In [1]:
# Importing Libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [ ]:
# Note:
#  -> Change the URL for each stock from https://in.investing.com
#  -> Change the file name while saving the dataset

In [13]:
# Setting Driver and Link
driver = webdriver.Chrome('D:/Data Analyst Course/Web scrapping/chromedriver_win32 (1)/chromedriver.exe')
url ='https://in.investing.com/equities/hdfc-bank-ltd-historical-data?end_date=1689273000&st_date=1560364200'
driver.get(url) 
driver.maximize_window() 
time.sleep(10) 

C:\Users\Dharmendra Kumar\AppData\Local\Temp\ipykernel_7584\4147941358.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('D:/Data Analyst Course/Web scrapping/chromedriver_win32 (1)/chromedriver.exe')


In [14]:
# Creating Soup
from bs4 import BeautifulSoup
content = driver.page_source
soup = BeautifulSoup(content)

In [15]:
# Date
date=[]
for a in soup.findAll('td', attrs={'class':'col-rowDate'}):
    date_txt = a.find('span', attrs={'class':'text'})
    date.append(date_txt.text)
#Closing Prices
close=[]
for a in soup.findAll('td', attrs={'class':'col-last_close'}):
    close_txt = a.find('span', attrs={'class':'text'})
    close.append(close_txt.text)
#Open Prices
open=[]
for a in soup.findAll('td', attrs={'class':'col-last_open'}):
    open_txt = a.find('span', attrs={'class':'text'})
    open.append(open_txt.text)
#High Prices
high=[]
for a in soup.findAll('td', attrs={'class':'col-last_max'}):
    high_txt = a.find('span', attrs={'class':'text'})
    high.append(high_txt.text)
#Low Prices
low=[]
for a in soup.findAll('td', attrs={'class':'col-last_min'}):
    low_txt = a.find('span', attrs={'class':'text'})
    low.append(low_txt.text)
#Volume
volume=[]
for a in soup.findAll('td', attrs={'class':'col-volume'}):
    volume_txt = a.find('span', attrs={'class':'text'})
    volume.append(volume_txt.text)
#Chnge%
chg=[]
for a in soup.findAll('td', attrs={'class':'col-change_percent'}):
    chg_txt = a.find('span', attrs={'class':'text'})
    chg.append(chg_txt.text)
    
volume = volume[:len(date)]
# Prepare DataFrame
df_nifty = pd.DataFrame({'Date':date,'Open':open,'High':high,'Low':low,'Close':close,'Volume':volume,'Chg%':chg})
df_nifty['Date'] = df_nifty['Date'].str.replace(r",","")
df_nifty['Date'] = pd.to_datetime(df_nifty.Date , format = '%b %d %Y')
df_nifty = df_nifty.drop_duplicates(subset="Date")               #droping dupicate data 
data = df_nifty

data['Close'] = data['Close'].str.replace(r",","").astype('float')
data['Open'] = data['Open'].str.replace(r",","").astype('float')
data['High'] = data['High'].str.replace(r",","").astype('float')
data['Low'] = data['Low'].str.replace(r",","").astype('float')

# data['Volume']=data['Volume'].astype(str)
# data['Chg%']=data['Chg%'].astype(str)
data

,Date,Open,High,Low,Close,Volume,Chg%
0,2023-07-13,1656.70,1657.45,1638.05,1641.10,20.96M,0.50%
1,2023-07-12,1655.35,1666.30,1627.15,1632.95,35.48M,-0.94%
2,2023-07-11,1663.00,1676.00,1645.50,1648.40,25.34M,-0.49%
3,2023-07-10,1661.00,1676.75,1649.70,1656.45,19.20M,-0.24%
4,2023-07-07,1665.00,1675.00,1656.30,1660.40,9.96M,-0.87%
...,...,...,...,...,...,...,...
1008,2019-06-19,1210.80,1219.73,1205.67,1211.48,5.37M,0.46%
1009,2019-06-18,1206.31,1213.30,1199.33,1205.94,6.21M,-0.20%
1010,2019-06-17,1214.80,1217.24,1205.97,1208.31,6.41M,-0.54%
1011,2019-06-14,1218.76,1226.74,1209.81,1214.85,5.98M,-0.39%


In [16]:
# Checking Null Values
data.isnull().sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
Chg%      0
dtype: int64

In [17]:
# Saving dataset with file_name
data.to_excel('E:/NIFTY50/HDFC Bank.xlsx', index=False)

### Data manipulation

In [9]:
def vol_change(x):
    y = str(x)
    if y[-1]=='M':
        a = y[:-1]
        return float(a)*1000000
    elif y[-1]=='K':
        a = y[:-1]
        return float(a)*1000

In [11]:
a='22.94K'
vol_change(a)

22940.0

In [ ]:
data['Volume']=data['Volume'].apply(lambda x: vol_change(x))

In [14]:
def remove_per(x):
    x = str(x)
    y = x[:-1]
    return float(y)

In [15]:
a='-0.28%'
remove_per(a)

-0.28

In [ ]:
data['Chg%']=data['Chg%'].apply(lambda x: remove_per(x))